In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.auto_mturk import (get_drive_service, download_forms_urls, 
                                 create_mturk_client,monitor_worker_tags,
                                 clean_own_worker,Turker)
from src.utils import generate_password
from src.constants import TOKEN_PATH,CREDS_PATH, FORMS_URLS_PATH, AWS_KEYS_PATH
from src.utils import read_access_keys
import pandas as pd
import jdc

In [17]:
# TOTRASH
import pytz
utc=pytz.UTC
from datetime import datetime
"""
Script to automate the creation of MTURK task: this assumes you did run the app script
generating the forms and their associate file previously.
"""
import io
import os.path
import pickle as pk
import shutil
import time
from datetime import datetime

import boto3
import pandas as pd
import pytz
import xmltodict
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from src.constants import (AWS_KEYS_PATH, CREDS_PATH, FORMS_URLS_PATH,
                           TOKEN_PATH, HIT2FORM_PATH)
from src.utils import generate_password, read_access_keys
from pathlib import Path

## Urls info retrieval

In [3]:
############ TODO: copy-paste the id of the most recent version of the url file ############
file_id = "1B1hGtXG2a-f7n4xit-EO0j5Bol2R8qh2Q1dp__MZKSU"
############################################################################################
# Retrieve the urls of the forms generated by the app script
service = get_drive_service()
download_forms_urls(FORMS_URLS_PATH,file_id,service)  # download the most recent forms_urls
forms_url = pd.read_csv(FORMS_URLS_PATH,sep=r"\s+",header=None,names=['url'],index_col=0)
forms_url = forms_url.url.to_dict()

Download 100%


## MTurk management

In [26]:
turk = Turker(hitlayout="3QS25971A2UW6PKAU5WN9771C2JXNI",
             MaxAssignments = 10,
             LifetimeInSeconds = 600,
             AutoApprovalDelayInSeconds=600,
             AssignmentDurationInSeconds=600,
             Reward='0.01',)

Loading hit2form


In [25]:
turk.create_forms_hits(forms_url)

Creating hit for form 0


AttributeError: 'Turker' object has no attribute 'HITLayoutId'

In [27]:
turk.list_hits()

Expiration:Nov 22 2020 16:03:13 (3.20 minutes left)


,HITId,Status,Completed,Percent_completed
FormIdx,,,,
0,3XWUWJ18TLBY3GYIH7CHK82TGCUUU5,Assignable,0,0
1,3KWGG5KP6JNJ4EC9ME0VZSZTWQKMCV,Assignable,0,0
2,3NSM4HLQNRFEJUKKJP87068FA6IQQR,Assignable,0,0
3,371DNNCG44NL56CGEZL2JKY207VT8X,Assignable,0,0
4,3A9LA2FRWSZLFYMTMDG5L0C7GM9XHZ,Assignable,0,0


In [11]:
clean_own_worker(turk.client)

RequestError: An error occurred (RequestError) when calling the DisassociateQualificationFromWorker operation: This operation can be called with a status of: Granted (1606055657826 s)

In [13]:
monitor_worker_tags(turk.client)

searching..
Tagging worker A29C1XYH77RQYM


KeyboardInterrupt: 

In [26]:
turk.list_all_results()

No results ready yet for 31JUPBOORNPXMT4B5QJBXTOL6U9L8M
No results ready yet for 3IH9TRB0FBKDLSCKL2DJQ39LBK9I1L
No results ready yet for 3QTFNPMJC632UP8JQTZ3N5PEAYGNZV


,WorkerId,HITId,Answer,Code,Status
0,A29C1XYH77RQYM,3YLTXLH3DFRG6PK20M6ORNQ4EN0PH3,932,932,Approved
0,A29C1XYH77RQYM,3MQKOF1EE29PEIC1QC1TSWJ2FSADWD,cako,125,Rejected


In [11]:
turk.list_results("3YLTXLH3DFRG6PK20M6ORNQ4EN0PH3")

,WorkerId,HITId,Answer,Code,Status
0,A29C1XYH77RQYM,3YLTXLH3DFRG6PK20M6ORNQ4EN0PH3,932,932,Approved


In [25]:
turk.approve_correct_hits()

Approving assignment 3K772S5NP9WFVIS6HAVFLUNG0SAEH7
Rejecting assignment 33FBRBDW6PK7597ZIFVEITUECX7C8X: given answer: cako correct password: 125


In [39]:
turk.delete_all_hits()

Hit 3P7RGTLO6EY0LBF8HOROEKH17IQAKY in Unassignable mode
Deleting hit 32FESTC2NHB2HMHIVGRHQ2GN9C9CUZ
Hit 3UYRNV2KITKNAT7WH3ZQALSABPV8NX in Unassignable mode


In [10]:
# reset the hits
turk.approve_all_hits()
turk.stop_all_hits()
turk.delete_all_hits()

Stop hit 3P4C70TRMR2CF8ATOOVZP65JUDMGLX
Stop hit 3ABAOCJ4R8PNYFMSRW1QJXOA05LQM4
Stop hit 322ZSN9Z5G5KM5PE2WVP28IXYLZT4Q
Stop hit 360ZO6N6J14H6G56AUQ4X2YLQ5HM9E
Stop hit 3D06DR52254VBZFB2044TX95WNZMAT
Hit 3P4C70TRMR2CF8ATOOVZP65JUDMGLX in Unassignable mode
Hit 3ABAOCJ4R8PNYFMSRW1QJXOA05LQM4 in Unassignable mode
Hit 322ZSN9Z5G5KM5PE2WVP28IXYLZT4Q in Unassignable mode
Hit 360ZO6N6J14H6G56AUQ4X2YLQ5HM9E in Unassignable mode
Hit 3D06DR52254VBZFB2044TX95WNZMAT in Unassignable mode


## Qualifications

In [187]:
# Create the qualification we need to distinguish workers who already performed an emoji task
turk.client.create_qualification_type(
    Name='emojidone',
    Description="Qualification to recognize workers that already performed an emoji-related task",
    QualificationTypeStatus='Active',
    AutoGranted=True,
    AutoGrantedValue=1
)
# QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN

RequestError: An error occurred (RequestError) when calling the CreateQualificationType operation: You have already created a QualificationType with this name. A QualificationType's name must be unique among all of the QualificationTypes created by the same user. (1605981881577 s)